In [1]:
import numpy as np
import pickle

import time
import math
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader

# from name_dataset import NameDataset
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.utils.data import Dataset, DataLoader

# Loading data

In [2]:
with open('/kaggle/input/urbansound8k-feature-extraction/train_data.pkl', 'rb') as f:
    X_train, Y_train = pickle.load(f)

with open('/kaggle/input/urbansound8k-feature-extraction/val_data.pkl', 'rb') as f:
    X_val, Y_val = pickle.load(f)


In [3]:
import pandas as pd
import numpy as np

def stack_samples(X, Y):
    """
    X: list or array of shape (N, R, C)
    Y: list of N labels
    Returns: DataFrame of shape (N×R, C + 1)
    """
    X_np = np.array(X, dtype=np.float32)  # Shape: (N, R, C)
    Y_repeated = np.repeat(Y, X_np.shape[1])  # Repeat each label R times

    X_stacked = X_np.reshape(-1, X_np.shape[2])  # (N×R, C)

    df = pd.DataFrame(X_stacked)
    df['labels'] = Y_repeated

    return df

train_df = stack_samples(X_train, Y_train)
val_df = stack_samples(X_val, Y_val)
print(train_df.shape)
print(val_df.shape)

(1173960, 181)
(355104, 181)


In [4]:
train_df.head(54)

,0,1,2,3,4,5,6,7,8,9,...,171,172,173,174,175,176,177,178,179,labels
0,0.027114,0.069684,0.084108,0.086272,0.086115,0.068998,0.075101,0.081550,0.082987,0.075951,...,0.078393,0.053319,0.0,0.0,0.0,0.0,0.0,0.0,0.0,jackhammer
1,0.430208,5.758408,51.152721,60.381077,21.515299,4.466881,13.074950,55.018288,43.365993,16.410397,...,25.460247,5.669981,0.0,0.0,0.0,0.0,0.0,0.0,0.0,jackhammer
2,0.356129,2.768868,10.043193,5.306677,2.211567,2.200262,5.633092,11.511503,4.667964,2.912486,...,5.367310,2.295357,0.0,0.0,0.0,0.0,0.0,0.0,0.0,jackhammer
3,0.119155,0.518129,1.104150,0.707675,0.361656,0.435153,0.531613,1.172129,1.274063,1.586210,...,1.393059,0.565431,0.0,0.0,0.0,0.0,0.0,0.0,0.0,jackhammer
4,0.200908,0.502827,0.919906,0.679136,0.617777,0.499163,0.740346,0.879904,0.768322,0.788436,...,1.237146,0.505317,0.0,0.0,0.0,0.0,0.0,0.0,0.0,jackhammer
5,0.170222,0.579005,0.849931,0.902989,0.607756,0.673738,0.697245,1.517281,1.158525,0.456944,...,0.783423,0.195463,0.0,0.0,0.0,0.0,0.0,0.0,0.0,jackhammer
6,0.110933,0.318491,1.234081,1.249484,1.085954,0.533614,0.550653,1.347195,0.960807,0.647482,...,0.694076,0.386585,0.0,0.0,0.0,0.0,0.0,0.0,0.0,jackhammer
7,0.164980,0.210829,0.551717,1.482775,2.704562,0.968102,0.979872,1.822335,2.469921,1.716560,...,1.440140,0.662305,0.0,0.0,0.0,0.0,0.0,0.0,0.0,jackhammer
8,0.080657,0.206064,0.383828,0.475435,0.290516,0.347667,0.401557,0.537649,0.564168,0.395445,...,1.213662,0.555193,0.0,0.0,0.0,0.0,0.0,0.0,0.0,jackhammer
9,0.067881,0.175669,0.380467,0.296372,0.141358,0.099105,0.360744,0.758478,0.346240,0.308535,...,0.190039,0.234406,0.0,0.0,0.0,0.0,0.0,0.0,0.0,jackhammer


In [5]:
val_df.head()

,0,1,2,3,4,5,6,7,8,9,...,171,172,173,174,175,176,177,178,179,labels
0,0.044180,0.056054,0.064415,0.069324,0.067334,0.067759,0.071800,0.069728,0.069248,0.065749,...,0.070372,0.061280,0.0,0.0,0.0,0.0,0.0,0.0,0.0,jackhammer
1,1.723533,5.687970,9.228014,7.410962,7.198935,11.340713,7.369151,10.589535,10.131034,3.039403,...,14.573446,10.166873,0.0,0.0,0.0,0.0,0.0,0.0,0.0,jackhammer
2,2.469047,7.855614,10.789010,9.705440,11.964504,14.177218,9.775521,12.749818,11.203400,4.147494,...,12.323687,5.084070,0.0,0.0,0.0,0.0,0.0,0.0,0.0,jackhammer
3,2.648386,5.412858,5.202138,6.334105,10.531077,8.599378,4.707143,6.276899,5.755446,8.934631,...,10.442652,4.127263,0.0,0.0,0.0,0.0,0.0,0.0,0.0,jackhammer
4,0.576799,1.618431,2.621299,1.711736,1.985173,2.103726,1.531987,2.888803,3.074646,3.813351,...,4.061364,3.374841,0.0,0.0,0.0,0.0,0.0,0.0,0.0,jackhammer


# Dataset Loader

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
BATCH_SIZE = 32
N_WORKERS = torch.cuda.device_count() if torch.cuda.device_count() > 1 else 1
N_WORKERS

2

In [8]:
label_map = {
    'air_conditioner': 0,
    'car_horn': 1,
    'children_playing': 2,
    'dog_bark': 3,
    'drilling': 4,
    'engine_idling': 5,
    'gun_shot': 6,
    'jackhammer': 7,
    'siren': 8,
    'street_music': 9
}

In [9]:
class UrbanSound8kDataset(Dataset):
    """ Diabetes dataset."""

    # Initialize your data, download, etc.
    def __init__(self, file_path):
        with open(file_path, 'rb') as f:
            self.x_data, self.y_data = pickle.load(f)
        self.len = len(self.x_data)
        self.y_data = np.array([label_map[label] for label in self.y_data])

    def __getitem__(self, idx):
        x = self.x_data[idx].astype(np.float32)   # <- fix here
        y = self.y_data[idx]
        # print(type(x[0][0]))
        # print(y)
        # print(torch.tensor(y, dtype=torch.long))
        return torch.from_numpy(x), torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return self.len


In [10]:
import numpy as np
import pickle

# Load train data to compute stats
with open('/kaggle/input/urbansound8k-feature-extraction/train_data.pkl', 'rb') as f:
    x_data, _ = pickle.load(f)

# Stack to shape (num_samples * frames, features) = (N * 180, 55)
x_all = np.vstack(x_data)
mean = x_all.mean(axis=0)     # shape: (55,)
std = x_all.std(axis=0) + 1e-8  # Add epsilon to avoid divide-by-zero

In [11]:
train_dataset = UrbanSound8kDataset('/kaggle/input/urbansound8k-feature-extraction/train_data.pkl')
train_loader = DataLoader(dataset=train_dataset,
                          batch_size=BATCH_SIZE,
                          shuffle=True,
                          num_workers=N_WORKERS)
val_dataset = UrbanSound8kDataset('/kaggle/input/urbansound8k-feature-extraction/val_data.pkl')
val_loader = DataLoader(dataset=val_dataset,
                          batch_size=BATCH_SIZE,
                          shuffle=True,
                          num_workers=N_WORKERS)


In [12]:
print(len(train_dataset), len(train_dataset[0]), len(train_dataset[0][0]), len(train_dataset[0][0][0]))

21740 2 54 180


In [13]:
print(len(train_dataset), len(train_dataset[1]), train_dataset[1][1])

21740 2 tensor(7)


In [14]:
print(len(val_dataset), len(val_dataset[0]), len(val_dataset[0][0]), len(val_dataset[0][0][0]))

6576 2 54 180


In [15]:
print(len(val_dataset), len(val_dataset[1]), val_dataset[1][1])

6576 2 tensor(7)


# RNN Model

In [154]:
class RNNClassifier(nn.Module):
    # Our model

    def __init__(self, input_size, hidden_size, output_size, n_layers=1, bidirectional=True):
        super(RNNClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        self.n_directions = int(bidirectional) + 1

        self.projection = nn.Linear(input_size, hidden_size)
        self.rnn1 = nn.RNN(hidden_size, hidden_size, n_layers,
                          bidirectional=bidirectional, batch_first=True)
        # self.rnn2 = nn.RNN(hidden_size * self.n_directions, hidden_size, n_layers,
        #                   bidirectional=bidirectional, batch_first=True)
        self.fc1 = nn.Linear(hidden_size * self.n_directions, hidden_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.2)
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, signal):
        # Note: we run this all at once (over the whole input sequence)
        # input shape: B x S (input size)
        # transpose to make S(sequence) x B (batch)
        # input = input.t()
        batch_size = signal.size(0)
        signal = signal.permute(0, 2, 1)

        # Make a hidden
        hidden = self._init_hidden(batch_size, signal.device)
        # print("hidden shape: ", hidden.shape)
        # print("signal shape: ", signal.shape)
        # # Embedding S x B -> S x B x I (embedding size)
        # embedded = self.embedding(input)

        # # Pack them up nicely
        # gru_input = pack_padded_sequence(
        #     embedded, seq_lengths.data.cpu().numpy())

        # To compact weights again call flatten_parameters().
        # self.gru.flatten_parameters()
        output, hidden = self.rnn1(self.projection(signal), hidden)
        # print("output shape: ", output.shape)
        # print("hidden shape: ", hidden.shape)
        # output, hidden = self.rnn2(output, hidden)

        if self.n_directions == 2:
            hidden = torch.cat((hidden[-2], hidden[-1]), dim=1)
        else:
            hidden = hidden[-1]
        
        # print("hidden shape: ", hidden.shape)
        # New
        hidden = self.fc1(hidden)
        hidden = self.relu(hidden)
        hidden = self.dropout(hidden)
        
        # Use the last layer output as FC's input
        # No need to unpack, since we are going to use hidden
        fc_output = self.fc2(hidden)
        # print("fc_output shape: ", fc_output.shape)
        
        return fc_output

    def _init_hidden(self, batch_size, device):
        hidden = torch.zeros(self.n_layers * self.n_directions,
                             batch_size, self.hidden_size).to(device)
        return create_variable(hidden)

In [174]:
class RNNClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers=2, bidirectional=True):
        super(RNNClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        self.n_directions = 2 if bidirectional else 1
        self.bidirectional = bidirectional

        self.projection = nn.Linear(input_size, hidden_size)
        self.rnn = nn.GRU(hidden_size, hidden_size, n_layers,
                          bidirectional=bidirectional, batch_first=True)
        
        self.fc1 = nn.Linear(hidden_size * self.n_directions, hidden_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.2)
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        batch_size = x.size(0)
        x = x.permute(0, 2, 1)  # Make it [B, S, I]

        
        h0 = torch.zeros(self.n_layers * self.n_directions, batch_size, self.hidden_size).to(x.device)
        out, hn = self.rnn(self.projection(x), h0)

        if self.bidirectional:
            hn = torch.cat((hn[-2], hn[-1]), dim=1)
        else:
            hn = hn[-1]

        x = self.fc1(hn)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)

        return x


In [29]:
# Train cycle
def train():
    total_loss = 0

    for i, (signal, label) in enumerate(train_loader, 1):
        output = classifier(signal)
        # print("signal size: ", signal.shape)
        # print("label size: ", label.shape)
        # print("label: ", label)
        loss = criterion(output, label)
        total_loss += loss.item()

        classifier.zero_grad()
        loss.backward()
        optimizer.step()

        if i % 100 == 0:
            print('[{}] Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.2f}'.format(
                time_since(start), epoch,  i *
                len(signal), len(train_loader.dataset),
                100. * i * len(signal) / len(train_loader.dataset),
                total_loss / i * len(signal)))

    return total_loss

In [30]:
def train():
    classifier.train()
    total_loss = 0

    for i, (signal, label) in enumerate(train_loader, 1):
        signal = signal.to(device)
        label = label.to(device)

        output = classifier(signal)
        loss = criterion(output, label)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * signal.size(0)

        if i % 100 == 0:
            avg_loss = total_loss / (i * signal.size(0))
            print('[{}] Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.4f}'.format(
                time_since(start), epoch, i * len(signal),
                len(train_loader.dataset),
                100. * i * len(signal) / len(train_loader.dataset),
                avg_loss))

    return total_loss / len(train_loader.dataset)


In [198]:
# Testing cycle
def test(name=None):
    
    print("evaluating trained model ...")
    correct = 0
    total_loss = 0
    train_data_size = len(val_loader.dataset)

    for signal, label in val_loader:
        output = classifier(signal)
        pred = output.data.max(1, keepdim=True)[1]
        total_loss += criterion(output, label)
        correct += pred.eq(label.data.view_as(pred)).cpu().sum()

    print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, train_data_size, 100. * correct / train_data_size))
    print(f"Loss: {total_loss / len(val_loader.dataset)}")


In [ ]:
def test(name=None):
    classifier.eval()
    correct = 0
    total = 0
    total_loss = 0

    with torch.no_grad():
        for signal, label in val_loader:
            signal = signal.to(device)
            label = label.to(device)
            

            output = classifier(signal)
            total_loss += criterion(output, label).item() * signal.size(0)
            pred = output.argmax(dim=1)
            correct += (pred == label).sum().item()
            total += label.size(0)

    accuracy = 100. * correct / total
    print('\nTest set: Accuracy: {}/{} ({:.2f}%)\n'.format(correct, total, accuracy))
    print(f"Loss: {total_loss / len(val_loader.dataset)}\n\n")

# Some utility functions

In [33]:
def create_variable(tensor):
    # Do cuda() before wrapping with variable
    if torch.cuda.is_available():
        return Variable(tensor.cuda())
    else:
        return Variable(tensor)

In [34]:
def time_since(since):
    s = time.time() - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

# Model Training & Valuation

In [155]:
N_EPOCHS = 100

N_INPUT = 54
HIDDEN_SIZE = 180
N_CLASSES = 10
N_LAYERS = 1


In [193]:
N_EPOCHS = 100

N_INPUT = 54
HIDDEN_SIZE = 256
N_CLASSES = 10
N_LAYERS = 2

classifier = RNNClassifier(
    input_size=N_INPUT,
    hidden_size=HIDDEN_SIZE,
    output_size=N_CLASSES,
    n_layers=N_LAYERS,
    # bidirectional=False
)


In [194]:
if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    # dim = 0 [33, xxx] -> [11, ...], [11, ...], [11, ...] on 3 GPUs
    classifier = nn.DataParallel(classifier)

Let's use 2 GPUs!


In [195]:
if torch.cuda.is_available():
    classifier.cuda()

In [207]:
test()


Test set: Accuracy: 3951/6576 (60.08%)

Loss: 1.757803818025148


In [209]:
optimizer = torch.optim.Adam(classifier.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

start = time.time()
print("Training for %d epochs..." % N_EPOCHS)
for epoch in range(1, N_EPOCHS + 1):
    # Train cycle
    train()

    # Testing
    test()

Training for 100 epochs...
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_35/705636435.py", line 8, in <cell line: 0>
    train()
  File "/tmp/ipykernel_35/983827614.py", line 13, in train
    loss.backward()
  File "/usr/local/lib/python3.11/dist-packages/torch/_tensor.py", line 626, in backward
    torch.autograd.backward(
  File "/usr/local/lib/python3.11/dist-packages/torch/autograd/__init__.py", line 347, in backward
    _engine_run_backward(
  File "/usr/local/lib/python3.11/dist-packages/torch/autograd/graph.py", line 823, in _engine_run_backward
    return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt

During handling of the above exception

TypeError: object of type 'NoneType' has no len()